# Temporal Node to Vec
Authors: Mohammad Ghassemi (ghassem3@msu.edu), Sanaz Hasanzadeh

## Step 1: Import Required Libraries

In [5]:
# NOTE!!! YOU HAVE TO RUN THIS CELL TWICE!
from math import ceil
from config import params
#from metrics import get_metrics
from utils import graph_utils
import loader
import models

In [6]:
#!pip install pandasql
import pandas   as pd
import pandasql as ps
import networkx as nx
import numpy    as np
import datetime as datetime

## Step 2: Import required Data

In [7]:
# First let's import the dataframe
df = pd.read_csv('data/people_data_u19.csv')
df

,pmid,pub_date,year,time_string,paper_post_2017,author_had_u19_in_2017,is_pi_a,name_key_a,papers_before_2017_a,papers_after_2017_a,is_pi_b,name_key_b,author_had_u19_in_2017_b,papers_before_2017_b,papers_after_2017_b
0,25565280,2015-01-21,2015,"datetime(2015,1,21,0,0)",0,0,0,G Broussard,3,13,0,R Liang,0,3,3
1,25565280,2015-01-21,2015,"datetime(2015,1,21,0,0)",0,0,1,L Tian,3,19,0,R Liang,0,3,3
2,25565280,2015-01-21,2015,"datetime(2015,1,21,0,0)",0,0,0,G Broussard,3,13,1,L Tian,0,3,19
3,25663667,2015-02-06,2015,"datetime(2015,2,6,0,0)",0,0,1,C Eroglu,20,36,0,W Chung,0,3,0
4,25663667,2015-02-06,2015,"datetime(2015,2,6,0,0)",0,0,0,N Allen,3,8,0,W Chung,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19592,30482694,2018-12-05,2018,"datetime(2018,12,5,0,0)",1,1,1,C Brody,5,17,1,D Tank,1,11,27
19593,30482694,2018-12-05,2018,"datetime(2018,12,5,0,0)",1,1,0,C Guo,4,2,1,D Tank,1,11,27
19594,30592258,2018-12-28,2018,"datetime(2018,12,28,0,0)",1,1,0,D Hampton,0,2,0,J Haley,1,0,2
19595,30592258,2018-12-28,2018,"datetime(2018,12,28,0,0)",1,1,0,E Marder,6,4,0,J Haley,1,0,2


## Step 3: Format the data

In [8]:
# Takes only the three columns we care about
graph_df = ps.sqldf("""SELECT pub_date as time,
                   name_key_a as id1,
                   name_key_b as id2
                 FROM df
                 LIMIT 10000
            """)

# Take the year from the timestamp column.
graph_df['time'] = pd.to_datetime(graph_df['time'])
cleansed         = [x.year for x in list(graph_df['time'])]
graph_df['time'] = cleansed 
graph_df

,time,id1,id2
0,2015,G Broussard,R Liang
1,2015,L Tian,R Liang
2,2015,G Broussard,L Tian
3,2015,C Eroglu,W Chung
4,2015,N Allen,W Chung
...,...,...,...
9995,2015,R Bruno,W Grueber
9996,2015,R Mann,W Grueber
9997,2015,V Voleti,W Grueber
9998,2015,C Lacefield,R Mann


## Step 4: Generate the Network X Graph

In [9]:
graph_nx = loader.dataset_loader.df2graph(graph_df, 'id1', 'id2', 'time', create_using=nx.Graph())

## Step 5: Generate Node Embeddings

In [10]:
tnodeembed = models.tNodeEmbed(graph_nx, task        = 'temporal_link_prediction', 
                                         dump_folder = 'results')

[2015 2016 2017 2018]
2018


Time Steps: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:39<00:00, 54.83s/time_step]

2018


In [11]:
X, y = tnodeembed.get_dataset()

-------------------------------------------
{'train': array([['G Broussard', 'C Wang'],
       ['G Broussard', 'D Miller'],
       ['C Wang', 'L Tian'],
       ...,
       ['E Driver-Dunckley', 'D Lo'],
       ['K Deisseroth', 'A Alsawadi'],
       ['K Karavitaki', 'M Wong']], dtype='<U21')}
{'train': array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])}
2017
-------------------------------------------


In [12]:
X['train'] = graph_utils.nodes2embeddings(X['train'], graph_nx         = tnodeembed.graph_nx, 
                                                      train_time_steps = tnodeembed.train_time_steps, 
                                                      dimensions       = 128)
tnodeembed.fit(X['train'] ,y['train'])


2021-08-24 17:56:46.262749: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-24 17:56:46.477465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


376/376 [==============================] - 4s 5ms/step - loss: 0.2647 - accuracy: 0.8900


In [13]:
X['test'] = graph_utils.nodes2embeddings(X['test'], graph_nx         = tnodeembed.graph_nx, 
                                                    train_time_steps = tnodeembed.train_time_steps,
                                                    dimensions       = 128)
tnodeembed.predict(X['test'])

array([[8.10248375e-01],
       [4.89331305e-01],
       [7.66489625e-01],
       [7.49334812e-01],
       [1.48103625e-01],
       [3.59228253e-03],
       [4.47629573e-05],
       [7.08552480e-01],
       [8.79136443e-01],
       [8.72773290e-01],
       [6.42558932e-03],
       [3.85724816e-05],
       [1.15755260e-01],
       [3.01733613e-03],
       [7.94624448e-01],
       [8.22482646e-01],
       [9.06964958e-01],
       [9.51051712e-04],
       [5.52385449e-02],
       [7.18424916e-01],
       [9.95907187e-01],
       [9.05771434e-01],
       [9.26985919e-01],
       [8.69903445e-01],
       [8.67331862e-01],
       [9.89279866e-01],
       [9.89034474e-01],
       [9.91048098e-01],
       [9.89032269e-01],
       [9.84875560e-01],
       [1.41209364e-03],
       [1.68052614e-02],
       [3.07601690e-03],
       [1.03604794e-03],
       [7.18317270e-01],
       [8.13902855e-01],
       [3.52677286e-01],
       [2.97261417e-01],
       [8.12672138e-01],
       [4.74539012e-01],
